In [4]:
!pip install transformers
!pip install torch
!pip install tensorflow

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached torch-1.13.1-cp37-cp37m-manylinux1_x86_64.whl (887.5 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvid

In [5]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")  # many other tasks are available
result = classifier("The actors were very convincing.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [10]:
classifier(["I am from Czech", "I am from Vietnam", "I am lesbian"])

[{'label': 'POSITIVE', 'score': 0.9564545750617981},
 {'label': 'NEGATIVE', 'score': 0.9747399091720581},
 {'label': 'NEGATIVE', 'score': 0.9941459894180298}]

In [62]:
# determine relationship - entailment, contradiction, neutral - MultiGenre NL Inference task
model_name = "huggingface/distilbert-base-uncased-finetuned-mnli"
classifier_mnli = pipeline("text-classification", model=model_name)
classifier_mnli("She loves me. [SEP] She loves me not.")

distiltunedmli_tokenizer = AutoTokenizer.from_pretrained(model_name)
distilberttunedmli_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


# distilbert-base-uncased

In [53]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
# AutoTokenizer loads the tokenizer for the 'distilbert-base-uncased' model. 
#TFAutoModelForSequenceClassification
#This model has been pre-trained on a large corpus of text. can be fine-tuned for sequence classification tasks.
distilbert_tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = TFAutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

# Sentiment analysis with labeled reviews - distilBERT

In [63]:
# head of data
import pandas as pd
import os

# Initialize a DataFrame to hold all the data
df = pd.DataFrame(columns=['review', 'sentiment'])

# Load the data from each .txt file
for filename in os.listdir('sentimentlabeled'):
    if filename.endswith('.txt'):
        path = os.path.join('sentimentlabeled', filename)
        data = pd.read_csv(path, sep='\t', names=['review', 'sentiment'])
        df = pd.concat([df, data])

# Display the first few rows
print(df.head())

                                              review sentiment
0  So there is no way for me to plug it in here i...         0
1                        Good case, Excellent value.         1
2                             Great for the jawbone.         1
3  Tied to charger for conversations lasting more...         0
4                                  The mic is great.         1


In [54]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

import numpy as np

# Tokenize the reviews
inputs = distilbert_tokenizer(df['review'].tolist(), truncation=True, padding='max_length', max_length=128, return_tensors='tf')

# Convert inputs to numpy arrays
input_ids = inputs['input_ids'].numpy()
attention_mask = inputs['attention_mask'].numpy()

# Convert sentiment labels to integers
labels = df['sentiment'].tolist()  # assuming the sentiment labels are already integers

# Split the data into training and validation sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, test_size=0.2)
train_masks, val_masks, _, _ = train_test_split(attention_mask, labels, test_size=0.2)

## DistilBERT-MLI

In [64]:

import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification

# Initialize the model
distiltunedmli_tokenizer = AutoTokenizer.from_pretrained(model_name)
distilberttunedmli_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Convert lists to tensors
train_labels = tf.convert_to_tensor(train_labels)
val_labels = tf.convert_to_tensor(val_labels)

# Compile the model sets the optimizer for the training process. 
#The optimizer is the algorithm used to change the attributes of the neural network such as weights and learning rate in order to reduce the losses. 
# Adam is a commonly used optimizer that adapts the learning rate for each weight individually. 
#The learning rate of `5e-5` is a commonly used value for transformer models.
distilberttunedmli_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
#sets the loss function, model will try to minimize during training.
# SparseCategoricalCrossentropy is a common loss function for multi-class classification problems.
# The `from_logits=True` argument means that the function should treat the model's output as unnormalized log probabilities (logits) rather than probabilities.
#`metrics=['accuracy']`: sets the metric evaluated during training and testing. Accuracy is a common metric for classification problems, and it simply measures the proportion of correct predictions.

# Train the model
distilberttunedmli_model.fit([train_inputs, train_masks], train_labels, validation_data=([val_inputs, val_masks], val_labels), batch_size=16, epochs=3)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Epoch 1/3
138/138 [==============================] - 57s 242ms/step - loss: 0.4046 - accuracy: 0.8212 - val_loss: 0.2786 - val_accuracy: 0.8709
Epoch 2/3
138/138 [==============================] - 31s 221ms/step - loss: 0.1748 - accuracy: 0.9308 - val_loss: 0.3625 - val_accuracy: 0.8727
Epoch 3/3
138/138 [==============================] - 30s 220ms/step - loss: 0.0742 - accuracy: 0.9732 - val_loss: 0.3882 - val_accuracy: 0.8618


In [54]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

import numpy as np

# Tokenize the reviews
inputs = distilbert_tokenizer(df['review'].tolist(), truncation=True, padding='max_length', max_length=128, return_tensors='tf')

# Convert inputs to numpy arrays
input_ids = inputs['input_ids'].numpy()
attention_mask = inputs['attention_mask'].numpy()

# Convert sentiment labels to integers
labels = df['sentiment'].tolist()  # assuming the sentiment labels are already integers

# Split the data into training and validation sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, test_size=0.2)
train_masks, val_masks, _, _ = train_test_split(attention_mask, labels, test_size=0.2)

## training from distilBert base 

In [55]:
# Data prepared, proceeding to fine tune distilBert
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification

# Initialize the model
distilbert_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Convert lists to tensors
train_labels = tf.convert_to_tensor(train_labels)
val_labels = tf.convert_to_tensor(val_labels)

# Compile the model sets the optimizer for the training process. 
#The optimizer is the algorithm used to change the attributes of the neural network such as weights and learning rate in order to reduce the losses. 
# Adam is a commonly used optimizer that adapts the learning rate for each weight individually. 
#The learning rate of `5e-5` is a commonly used value for transformer models.
distilbert_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
#sets the loss function, model will try to minimize during training.
# SparseCategoricalCrossentropy is a common loss function for multi-class classification problems.
# The `from_logits=True` argument means that the function should treat the model's output as unnormalized log probabilities (logits) rather than probabilities.
#`metrics=['accuracy']`: sets the metric evaluated during training and testing. Accuracy is a common metric for classification problems, and it simply measures the proportion of correct predictions.

# Train the model
distilbert_model.fit([train_inputs, train_masks], train_labels, validation_data=([val_inputs, val_masks], val_labels), batch_size=16, epochs=3)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
138/138 [==============================] - 58s 244ms/step - loss: 0.3996 - accuracy: 0.8057 - val_loss: 0.3015 - val_accuracy: 0.8582
Epoch 2/3
138/138 [==============================] - 30s 221ms/step - loss: 0.1934 - accuracy: 0.9240 - val_loss: 0.3045 - val_accuracy: 0.8600
Epoch 3/3
138/138 [==============================] - 31s 221ms/step - loss: 0.1060 - accuracy: 0.9627 - val_loss: 0.4039 - val_accuracy: 0.8564


In [69]:
# Save the fine-tuned DistilBERT model, checkpoint
distilbert_model.save_pretrained('tony_tuned_distilbert_model')
distilbert_tokenizer.save_pretrained('tony_tuned_distilbert_tokenizer')

distilberttunedmli_model.save_pretrained('tunedmli_model')
distiltunedmli_tokenizer.save_pretrained('tunedmli_tokenizer')

('tunedmli_tokenizer/tokenizer_config.json',
 'tunedmli_tokenizer/special_tokens_map.json',
 'tunedmli_tokenizer/vocab.txt',
 'tunedmli_tokenizer/added_tokens.json',
 'tunedmli_tokenizer/tokenizer.json')

## Load checkpoint


In [ ]:
# Load checkpoint
distilbert_model = TFDistilBertForSequenceClassification.from_pretrained('tony_tuned_distilbert_model')
distilbert_tokenizer = AutoTokenizer.from_pretrained('tony_tuned_distilbert_tokenizer')

Some layers from the model checkpoint at tony_tuned_distilbert_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_377']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at tony_tuned_distilbert_model and are newly initialized: ['dropout_417']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load checkpoint
distilberttunedmli_model.save_pretrained('tunedmli_model')
distiltunedmli_tokenizer.save_pretrained('tunedmli_tokenizer')

Some layers from the model checkpoint at tony_tuned_distilbert_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_377']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at tony_tuned_distilbert_model and are newly initialized: ['dropout_417']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Model eval - tuned mli

In [85]:
# Suppose you have a new review
new_review = ["The hotel was excellent!", "The room was very clean.", "The food was terrible.", "I wouldn't recommend this place.", "Best hotel I've ever stayed at!"]
# Make sure the review is a list of strings
if isinstance(new_review, str):
    new_review = [new_review]
    
# Tokenize the new review
new_inputs = distiltunedmli_tokenizer(new_review, truncation=True, padding=True, return_tensors='tf')

# Make a prediction
predictions = distilberttunedmli_model.predict([new_inputs['input_ids'], new_inputs['attention_mask']])

# The predictions are logits (unnormalized scores). softmax function, to get probabilities
probabilities = tf.nn.softmax(predictions[0], axis=-1)

# Print the probabilities
print(probabilities)

1/1 [==============================] - 0s 44ms/step
tf.Tensor(
[[1.2880618e-03 9.9847311e-01 2.3870940e-04]
 [1.8707142e-03 9.9784184e-01 2.8737498e-04]
 [9.9959046e-01 2.3521387e-04 1.7433712e-04]
 [9.9702990e-01 2.4592008e-03 5.1087979e-04]
 [2.4574117e-03 9.9711823e-01 4.2431121e-04]], shape=(5, 3), dtype=float32)


#### fine-tuned on the Multi-Genre Natural Language Inference (MNLI) task. The MNLI task is a three-class classification problem, where each instance is classified as entailment, contradiction, or neutral. 

In [72]:
from sklearn.metrics import roc_auc_score

# Get the model's predicted probabilities for the positive class
probabilities = distilberttunedmli_model.predict([val_inputs, val_masks])[0]
probabilities = tf.nn.softmax(probabilities, axis=-1)
positive_probabilities = probabilities[:, 1]

# Compute the AUC-ROC
auc_roc = roc_auc_score(val_labels, positive_probabilities)

print('AUC-ROC:', auc_roc)

18/18 [==============================] - 4s 131ms/step
AUC-ROC: 0.952811404906021


In [73]:
from sklearn.metrics import f1_score
import numpy as np

# Get the model's predicted probabilities
probabilities = distilberttunedmli_model.predict([val_inputs, val_masks])[0]
probabilities = tf.nn.softmax(probabilities, axis=-1)

# Convert probabilities to class predictions
class_predictions = np.argmax(probabilities, axis=-1)

# Compute the F1 score - harmonic mean of precision and recall
f1 = f1_score(val_labels, class_predictions, average='weighted')

print('F1 Score:', f1)

18/18 [==============================] - 2s 133ms/step
F1 Score: 0.8606480938416422


## Model eval finetuned distilbert_model

### class `0` for negative sentiment and class `1` for positive sentiment. 
the model is predicting with high confidence (approximately 99.63%) that the review "The hotel was excellent!" has a positive sentiment. 
The low probability (approximately 0.37%) corresponds to the negative sentiment.

In [ ]:
# Suppose you have a new review
new_review = ["The hotel was excellent!", "The room was very clean.", "The food was terrible.", "I wouldn't recommend this place.", "Best hotel I've ever stayed at!"]
# Make sure the review is a list of strings
if isinstance(new_review, str):
    new_review = [new_review]
    
# Tokenize the new review
new_inputs = distilbert_tokenizer(new_review, truncation=True, padding=True, return_tensors='tf')

# Make a prediction
predictions = distilbert_model.predict([new_inputs['input_ids'], new_inputs['attention_mask']])

# The predictions are logits (unnormalized scores). softmax function, to get probabilities
probabilities = tf.nn.softmax(predictions[0], axis=-1)

# Print the probabilities
print(probabilities)

1/1 [==============================] - 0s 46ms/step
tf.Tensor(
[[0.0036537  0.9963463 ]
 [0.9934685  0.00653151]
 [0.9972819  0.00271812]
 [0.06982092 0.93017906]
 [0.00551693 0.99448305]], shape=(5, 2), dtype=float32)


notice `room was very clean` is scored as negative. False negative

In [74]:
from sklearn.metrics import roc_auc_score

# Get the model's predicted probabilities for the positive class
probabilities = distilbert_model.predict([val_inputs, val_masks])[0]
probabilities = tf.nn.softmax(probabilities, axis=-1)
positive_probabilities = probabilities[:, 1]

# Compute the AUC-ROC
auc_roc = roc_auc_score(val_labels, positive_probabilities)

print('AUC-ROC:', auc_roc)

18/18 [==============================] - 4s 130ms/step
AUC-ROC: 0.9463204842306466


In [76]:
from sklearn.metrics import f1_score
import numpy as np

# Get the model's predicted probabilities
probabilities = distilbert_model.predict([val_inputs, val_masks])[0]
probabilities = tf.nn.softmax(probabilities, axis=-1)

# Convert probabilities to class predictions
class_predictions = np.argmax(probabilities, axis=-1)

# Compute the F1 score - harmonic mean of precision and recall
f1 = f1_score(val_labels, class_predictions, average='weighted')

print('F1 Score:', f1)

18/18 [==============================] - 2s 131ms/step
F1 Score: 0.8563470172253012


# Electra model 


In [78]:
from transformers import TFElectraForSequenceClassification, ElectraTokenizer

# Initialize the tokenizer and the model
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')
electra_model = TFElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator')

# Convert lists to tensors
train_labels = tf.convert_to_tensor(train_labels)
val_labels = tf.convert_to_tensor(val_labels)

# Compile the model
electra_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

# Train the model
electra_model.fit([train_inputs, train_masks], train_labels, validation_data=([val_inputs, val_masks], val_labels), batch_size=16, epochs=3)

Some layers from the model checkpoint at google/electra-base-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
138/138 [==============================] - 112s 466ms/step - loss: 0.4331 - accuracy: 0.7925 - val_loss: 0.2036 - val_accuracy: 0.9073
Epoch 2/3
138/138 [==============================] - 60s 437ms/step - loss: 0.2292 - accuracy: 0.9140 - val_loss: 0.3036 - val_accuracy: 0.8873
Epoch 3/3
138/138 [==============================] - 60s 434ms/step - loss: 0.1424 - accuracy: 0.9540 - val_loss: 0.3933 - val_accuracy: 0.9145


In [80]:
# Save the fine-tuned ELECTRA model
electra_model.save_pretrained('electra_model')
electra_tokenizer.save_pretrained('electra_tokenizer')

('electra_tokenizer/tokenizer_config.json',
 'electra_tokenizer/special_tokens_map.json',
 'electra_tokenizer/vocab.txt',
 'electra_tokenizer/added_tokens.json')

### Load Electra checkpoint


In [ ]:
electra_tokenizer = ElectraTokenizer.from_pretrained('electra_tokenizer')
electra_model = TFElectraForSequenceClassification.from_pretrained('electra_model')

In [81]:
#predict
new_review = ["The hotel was excellent!", "The room was very clean.", "The food was terrible.", "I wouldn't recommend this place.", "Best hotel I've ever stayed at!"]

# Make sure the review is a list of strings
if isinstance(new_review, str):
    new_review = [new_review]

# Tokenize the new review
new_inputs = electra_tokenizer(new_review, truncation=True, padding=True, return_tensors='tf')

# Make a prediction
predictions = electra_model.predict([new_inputs['input_ids'], new_inputs['attention_mask']])

# The predictions are logits (unnormalized scores). Apply the softmax function to get probabilities
probabilities = tf.nn.softmax(predictions[0], axis=-1)

# Print the probabilities
print(probabilities)

1/1 [==============================] - 3s 3s/step
tf.Tensor(
[[0.0030535  0.9969465 ]
 [0.00287638 0.99712366]
 [0.97472817 0.02527181]
 [0.96255565 0.0374443 ]
 [0.00304213 0.9969579 ]], shape=(5, 2), dtype=float32)


## Model evaluation tuned Elektra 

In [86]:
from sklearn.metrics import roc_auc_score

# Get the model's predicted probabilities for the positive class
probabilities = electra_model.predict([val_inputs, val_masks])[0]
probabilities = tf.nn.softmax(probabilities, axis=-1)
positive_probabilities = probabilities[:, 1]

# Compute the AUC-ROC
auc_roc = roc_auc_score(val_labels, positive_probabilities)

print('AUC-ROC:', auc_roc)

18/18 [==============================] - 8s 265ms/step
AUC-ROC: 0.9665233089094192


In [89]:
from sklearn.metrics import f1_score

# Get the model's predictions
predictions = electra_model.predict([val_inputs, val_masks])[0]
predictions = tf.nn.softmax(predictions, axis=-1)

# Convert the probabilities to class labels
predicted_labels = tf.argmax(predictions, axis=1)

# Compute the F1 score
f1 = f1_score(val_labels, predicted_labels, average='weighted')

print('F1 Score:', f1)

18/18 [==============================] - 5s 261ms/step
F1 Score: 0.9137907653583428
